PAIR DISCOVERY - CLUSTERING AND COINTEGRATION

This section identifies tradable pairs within each sector using a two step statistical approach

Time period:
1.training data only (2021-01-01 to 2023-12-31)
2.test period (2024-2024) is reserved for out-of-sample validation

processing steps:
1.load price and returns data for each sector
2.apply time filter to use only training period data
3.cluster stocks based on return similarities
4.test for cointegration between all stock pairs within each cluster
5.collect pairs that pass the cointegration test

OUTPUT: "01_cointegrated_pairs.csv in /data/processed/
-stock1, stock2, (the pairs)
-cluster ID
sector
-cointegration test statistics


import from src.statitics the following functions:
1.get_clusters
2.find_cointegrated_pairs

PCA CONFIGURATION
A fixed value of 5 principal components was used across all sectors for consistency.$

The following code performs a diagnostic analysis, showing the optimal component vary by sector. While a dynamic approach could improve clustering accuracy, a fixed value was choosen to maintain a uniform methology and prioritize dominant market factors

In [12]:
# PCA DIAGNOSTIC - All sectors

import os
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

processed_dir = '../data/processed'

for filename in os.listdir(processed_dir):
    if filename.endswith('_returns.csv'):
        sector = filename.replace('_returns.csv', '')
        
        returns_df = pd.read_csv(os.path.join(processed_dir, filename), 
                                  index_col=0, parse_dates=True)
        
        pca = PCA(n_components=min(15, len(returns_df.columns)-1))
        pca.fit(returns_df)
        
        cumvar = pca.explained_variance_ratio_.cumsum()
        n_for_80 = (cumvar >= 0.80).argmax() + 1
        
        print(f"{sector}: {n_for_80} components needed for 80% variance")

communication_services: 7 components needed for 80% variance
consumer_discretionary: 12 components needed for 80% variance
consumer_staples: 11 components needed for 80% variance
energy: 3 components needed for 80% variance
financials: 9 components needed for 80% variance
health_care: 1 components needed for 80% variance
industrials: 1 components needed for 80% variance
information_technology: 14 components needed for 80% variance
materials: 7 components needed for 80% variance
real_estate: 8 components needed for 80% variance
utilities: 5 components needed for 80% variance


In [18]:
#FIND TRADABLE RELATIONSHPS
#TRADABLE PAIRS

import pandas as pd
import os
import sys

# Setup paths
sys.path.append(os.path.abspath(os.path.join('..')))
from src.statistics import get_clusters, find_cointegrated_pairs

raw_dir = '../data/raw'
processed_dir = '../data/processed'

#time filter configuration
TRAIN_START_DATE = pd.Timestamp('2021-01-01')
TRAIN_END_DATE = pd.Timestamp('2023-12-31')



all_valid_pairs = []

print("PART 1: MINING FOR PAIRS (Clustering and Cointegration)")
print(f"TIME FILTER ACTIVE: Only analyzing data from {TRAIN_START_DATE.date()} to {TRAIN_END_DATE.date()}")
print(f"(Training period: 2021-2023 | Test period: 2024-2025)\n")


for filename in os.listdir(raw_dir):
    if filename.endswith("_prices.csv"):
        sector_name = filename.replace("_prices.csv", "")
        print(f"\n--- Sector: {sector_name.upper()} ---")

        # Load Data
        try:
            price_path = os.path.join(raw_dir, filename)
            returns_path = os.path.join(processed_dir, f"{sector_name}_returns.csv")
            
            if not os.path.exists(returns_path):
                print("   Missing returns file. Skipping.")
                continue

            prices = pd.read_csv(price_path, index_col=0, parse_dates=True)
            returns = pd.read_csv(returns_path, index_col=0, parse_dates=True)
        except Exception as e:
            print(f"   Load Error: {e}")
            continue

#apply time filter
        prices = prices.loc[TRAIN_START_DATE:TRAIN_END_DATE]
        returns = returns.loc[TRAIN_START_DATE:TRAIN_END_DATE]

        # Step 1: Cluster
        print("   Clustering...")
        found_clusters = get_clusters(returns, eps=0.35)
        if found_clusters.empty: continue

        # Step 2: Cointegration
        print("   Cointegration Test...")
        sector_pairs = find_cointegrated_pairs(prices, found_clusters)

        if not sector_pairs.empty:
            # Clean up names
            sector_pairs = sector_pairs.rename(columns={'Stock A': 'Stock1', 'Stock B': 'Stock2'})
            sector_pairs['Cluster'] = sector_name + "_" + sector_pairs['Cluster'].astype(str)
            sector_pairs['Sector'] = sector_name
            
            all_valid_pairs.append(sector_pairs)
            print(f"   -> Found {len(sector_pairs)} cointegrated pairs.")

# SAVE CHECKPOINT
if all_valid_pairs:
    master_list = pd.concat(all_valid_pairs, ignore_index=True)
    output_path = '../data/processed/01_cointegrated_pairs.csv'
    master_list.to_csv(output_path, index=False)
    print(f"\nSUCCESS! Saved checkpoint to: {output_path}")
else:
    print("\nNo pairs found.")

PART 1: MINING FOR PAIRS (Clustering and Cointegration)
TIME FILTER ACTIVE: Only analyzing data from 2021-01-01 to 2023-12-31
(Training period: 2021-2023 | Test period: 2024-2025)


--- Sector: COMMUNICATION_SERVICES ---
   Clustering...
   Cointegration Test...
Testing Cointegration on 2 clusters...
  > Checking Cluster 0 (3 stocks)...
  > Checking Cluster 1 (2 stocks)...
   -> Found 2 cointegrated pairs.

--- Sector: CONSUMER_DISCRETIONARY ---
   Clustering...
   Cointegration Test...
Testing Cointegration on 8 clusters...
  > Checking Cluster 0 (2 stocks)...
  > Checking Cluster 1 (8 stocks)...
  > Checking Cluster 2 (3 stocks)...
  > Checking Cluster 3 (6 stocks)...
  > Checking Cluster 4 (2 stocks)...
  > Checking Cluster 5 (2 stocks)...
  > Checking Cluster 6 (4 stocks)...
  > Checking Cluster 7 (3 stocks)...
   -> Found 4 cointegrated pairs.

--- Sector: CONSUMER_STAPLES ---
   Clustering...
   Cointegration Test...
Testing Cointegration on 3 clusters...
  > Checking Cluster 0 (

In [19]:
#save the cointegrated pairs table to outputs folder as well
import os

# 1. Define the paths
source_path = '../data/processed/02_final_tradable_pairs.csv'
output_dir = '../results'
output_path = os.path.join(output_dir, 'final_tradable_pairs.csv')

# 2. Load the CSV 
df_final = pd.read_csv(source_path)

# 4. Save the table to the output folder
df_final.to_csv(output_path, index=False)

print(f"Table successfully saved to: {output_path}")

Table successfully saved to: ../results\final_tradable_pairs.csv


In [20]:
#check if all GICS sectors were processed, or if not which sector were processed
master_list = pd.read_csv('../data/processed/01_cointegrated_pairs.csv')
print(master_list['Sector'].unique())

['communication_services' 'consumer_discretionary' 'consumer_staples'
 'energy' 'financials' 'health_care' 'industrials'
 'information_technology' 'materials' 'utilities']
